# 乗除演算
一般の足し算と引き算の仕組みを利用してあまりを求めようと思います。

回路は以下のようになります。

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2Ff161e722-f727-b6fb-d02b-24e40b5b63b1.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=da298eace648814b5ccfe4affa9301ca">

ここで $0 < a,b < N$ とします。
最後のビットは overflow を確認するもので temporary bit と呼ばれます。

## 実装手順
$a+b \mod N$ を求めるためには $a+b$ と $N$ の大小を比較する必要があります。

$a+b>N$ のとき  
$0<a,b<N$ より $0<a+b<2N$ よって $0<a+b-N<N$ より $a+b-N = (a+b) \mod N$

$a+b<N$ のとき  
$a+b = (a+b) \mod N$

これを量子回路で行います。

$a+b>N$

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F5c97b6fd-425b-c42b-057a-68d9cb47e457.jpeg?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=3711f239a543dc4aa175f4636adde888">

$a+b<N$

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2Fcb7bc1bf-e713-f7d4-9cdc-4bc47bb6eba8.jpeg?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=425f2f86ef9d5b5c27b1c1f1ddfab936">

余剰の量子ビットと加算器の最上位の量子ビットの値を使ってうまく場合分けをしています。a+b<N の場合には余剰ビットを使わず余計な操作もありません。

## 例題
簡単な例題を見てみます。

$3 + 5 < 7$ のとき  
$(3 + 5) \mod 7 = 1$

$3 + 5 > 11$ のとき  
$(3 + 5) \mod 11 = 8$

これを量子回路を使って実現しようというのが今回の剰余演算です。

## Blueqatのインストール
pipからBlueqatをインストールします。

In [1]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


### 実装
実装するために下準備をします。

今回は上の回路に N をもう１つ加えて temporary bit と N を対応させます。  
初期状態を以下のようにします。

```
c0 --
a0 --
b0 --
c1 --
..
n0 --
n1 --
..
t  --
n0 --
n1 --
..
```

また各桁は $0<a,b<N$ より $N$ に統一します。 

In [118]:
from blueqat import Circuit

#ビットのキャリー回路
def carry(a):
    return Circuit().ccx[a+1,a+2,a+3].cx[a+1,a+2].ccx[a,a+2,a+3]

#ビットのキャリー回路の逆
def carry_reverse(a):
    return Circuit().ccx[a,a+2,a+3].cx[a+1,a+2].ccx[a+1,a+2,a+3]

#ビットの合計
def sum(a):
    return Circuit().cx[a+1,a+2].cx[a,a+2] 

#ビットの合計の逆
def sum_reverse(a):
    return Circuit().cx[a,a+2].cx[a+1,a+2]

#10進数を2進数に
def tobinary(A):
    return bin(A)[2:] 

#3つの10進数を2進数に直して、桁を揃えてモジュロ回路の順にビットを並べ替える。一番下に判定用のビットを１つ加える。
def digits2(a,b,n): 
     aa = tobinary(a)  
     bb = tobinary(b)  
     nn = tobinary(n)  

     nlen = len(nn)  
     aa = aa.zfill(nlen) 
     bb = bb.zfill(nlen) 

     str = '' 
     for i in range(nlen): 
         str += '0' + aa[nlen-i-1] + bb[nlen-i-1] 
     str += '0' 

     for i in range(nlen): 
         str += nn[nlen-i-1]  

     str += '0'

     for i in range(nlen): 
        str += nn[nlen-i-1]

     return str

#ビット文字列をXゲートを使ったデータ入力回路に変換
def toX(a): 
     cir = Circuit(len(a)) 
     for i in range(len(a)): 
         if a[i] == "1": 
             cir += Circuit().x[i] 
     return cir

#足し算回路
def plus(a,b,n): 
     qubits = len(digits2(a,b,n))
     digi = len(tobinary(n))

     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry(i*3) 

     cir3 = Circuit(qubits).cx[(digi-1)*3+1,(digi-1)*3+2] + sum((digi-1)*3)

     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += carry_reverse((digi-i-2)*3) 
         cir4 += sum((digi-i-2)*3)

     cir_plus = cir2 + cir3 + cir4
     return cir_plus

#引き算回路
def minus(a,ab,n): 
     qubits = len(digits2(a,ab,n))
     digi = len(tobinary(n))

     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += sum_reverse(i*3)
         cir4 += carry(i*3) 

     cir3 = sum_reverse((digi-1)*3) + Circuit(qubits).cx[(digi-1)*3+1,(digi-1)*3+2] 

     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry_reverse((digi-1-i)*3) 

     cir_minus = cir4 + cir3 + cir2
     return cir_minus

#aとNを交換
def swap(n):
    digi = len(tobinary(n))
    cir = Circuit(5*digi+2)
    for i in range(digi):
        cir += Circuit(5*digi+2).swap[3*i+1,3*digi+1+i]
    return cir

#2進数を10進数に
def todecimal(A):
    return int(str(A),2) 

#回路から解だけを抜き出す
def getb(result,n): 
     str = ""
     digi = len(tobinary(n)) 
     for i in range(digi): 
         str += result[3*(digi-i)-1]
     return todecimal(str)

### 一般化回路
以上で全ての準備が整ったので剰余演算回路を作ります。

In [119]:
def adder_mod(a,b,n):
    digi = len(tobinary(n))

    # 最初の部分
    part1 = toX(digits2(a,b,n)) + plus(a,b,n) + swap(n) + minus(a,b,n)

    # overflow を temporary bit に格納
    part2 = Circuit(5*digi+2).x[digi*3].cx[digi*3,digi*4+1].x[digi*3]

    # temporary bit で N を返す
    part3 = Circuit(5*digi+2)
    for i in range(digi):
        part3 += Circuit(5*digi+2).ccx[4*digi+2+i,4*digi+1,3*i+1]

    # 最後の部分
    part4 = minus(a,b,n)+Circuit(5*digi+2).cx[digi*3,digi*4+1]+plus(a,b,n)
    
    result = (part1+part2+part3+plus(a,b,n)+part3+swap(n)+part4).m[:].run(shots=1)
    return getb(result.most_common()[0][0],n)

実際に計算してみます。

In [120]:
adder_mod(4,3,5)

2

In [121]:
adder_mod(4,4,5)

3

In [122]:
adder_mod(1,5,6)

0

一般の剰余演算回路を実装できました。

### 参考文献
V. Vedral, A. Barenco, A. Ekert, "Quantum Networks for Elementary Arithmetic Operations",  
(Submitted on 16 Nov 1995)  
https://arxiv.org/pdf/quant-ph/9511018.pdf